# Part I. ETL Pipeline for Pre-Processing the Files

## FILE PRE-PROCESSING

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Print total number of rows if needed
#print(len(full_data_rows_list))

# Print entire list of event data rows if needed
#print(full_data_rows_list))

# Create a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra 

## The <font color=red>event_datafile_new.csv</font> located within the Workspace directory contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in cells below

#### Creating a Cluster

In [5]:
# Makes a connection to a Cassandra instance locally (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Establish connection and begin executing queries by assigning session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Next, I create tables to run the following 3 queries. The database will be modelled with Apache Cassandra based on the queries that I need to run.

### Below are 3 questions to be answered with 3 separate queries.

#### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

Since we need the artist, song title, and length, these 3 columns would need to be columns in `session_table`.  Also since we are filtering by sessionId and itemInSession, they would need to be columns in the table as well.  The primary key consists of partition key `sessionId` and clustering column `itemInSession` because `sessionId` would be unique for each session and `itemInSession` would make each song that the user listens to unique in my query.  Each column is assigned an appropriate data type, such as text, int, and float.

In [8]:
## Query 1:  Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

query1 = "CREATE TABLE IF NOT EXISTS session_table (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query1)
except Exception as e:
    print(e)
                    

In [9]:
# Load data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

# Assign the INSERT statements into the `query1` variable
        
        query1 = "INSERT INTO session_table (sessionId, itemInSession, artist, song, length) VALUES (%s, %s, %s, %s, %s)"
        
# Line below assigns which column element should be assigned for each column in the INSERT statement.
# For example, to artist_name and user first_name, would correspond to `line[0], line[1]`
        
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Execute SELECT statement to get the results of the query

query1 = "SELECT artist, song, length FROM session_table WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

#for row in rows:
#    print (row.artist, row.song, row.length)

# Convert above result into pandas dataframe and display query results
df_query1 = pd.DataFrame(list(rows))
df_query1

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2


The primary key consists of `sessionId` as the partition key, along with `userId` and `itemInSession` as clustering columns, because we are filtering by userId=10 and sessionId=182.  The combination of partition key and the 2 cluster columns would be unique in the table.  `itemInSession` as a clustering column would sort the items in the query.  Each column is assigned an appropriate data type, such as text and int.

In [11]:
## Query 2: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query2 = """CREATE TABLE IF NOT EXISTS user_playlist (
            sessionId int, 
            userId int, 
            itemInSession int, 
            artist text, 
            song text, 
            firstName text, 
            lastName text, 
            PRIMARY KEY (sessionId, userId, itemInSession))"""

try:
    session.execute(query2)
except Exception as e:
    print(e)

In [12]:
# Load data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

# Assign the INSERT statements into the `query2` variable
        query2 = "INSERT INTO user_playlist (sessionId, userId, itemInSession, artist, song, firstName, lastName) VALUES (%s, %s, %s, %s, %s, %s, %s)"    
        session.execute(query2, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# Execute SELECT statement to get the results of the query

query2 = "SELECT artist, song, firstName, lastName FROM user_playlist WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
#for row in rows:
#    print (row.artist, row.song, row.firstname, row.lastname)
    
# Convert above result into pandas dataframe and display query results
df_query2 = pd.DataFrame(list(rows))
df_query2

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3

Primary key is comprised of `song` as the partition key and `userId` as the clustering column.  For this particular query, the primary key combination would be unique. Each column is assigned an appropriate data type, such as text and int.

In [14]:
## Query 3: Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3 = "CREATE TABLE IF NOT EXISTS song_select_users (song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
        
try:
    session.execute(query3)
except Exception as e:
    print(e)             

In [15]:
# Load data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## Assign the INSERT statements into the `query3` variable
        query3 = "INSERT INTO song_select_users (song, userId, firstName, lastName) VALUES (%s, %s, %s, %s)"         
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))
   

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
# Execute SELECT statement to get the results of the query

query3 = "SELECT firstName, lastName FROM song_select_users WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
#for row in rows:
#    print (row.firstname, row.lastname)
    
# Convert above result into pandas dataframe and display query results
df_query3 = pd.DataFrame(list(session.execute(query3)))
df_query3

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
try:
    rows = session.execute("DROP TABLE session_table")
    rows = session.execute("DROP TABLE user_playlist")
    rows = session.execute("DROP TABLE song_select_users")
except Exception as e:
    print(e)

### Close the session and cluster connection

In [18]:
session.shutdown()
cluster.shutdown()